In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

import pandas as pd
from sqlalchemy import create_engine
import pymysql
import numpy as np
import plotly.express as px
from adj_close import adj_close
from tqdm import tqdm
pw='baadf00d'

db = pymysql.connect(host='192.168.148.52', port=3306, user='admin', passwd=pw, db='data', charset='utf8')
ks="SELECT * FROM data.price WHERE date >= '2018-06-15'; "
curs=db.cursor(pymysql.cursors.DictCursor)


curs.execute(ks)


item_data=curs.fetchall()
item_data=pd.DataFrame(item_data)
item_data


from adj_close import adj_close
price=adj_close(item_data)
price['adj_close']=price['adj_close'].apply(lambda x: np.round(x))


In [2]:
import statsmodels.api as sm

price=price.loc[(price['market']!='외감') & (price['market']!='KONEX')]
###거래정지 종목 제거
price=price.loc[price['trading_suspension']!=1]
##상장종목수 0인거 제거
price['listed_stocks'].fillna(0,inplace=True)
price=price.loc[price['listed_stocks']!=0]
price['rtn']=np.log(price.groupby('code').adj_close.shift(0)/price.groupby('code').adj_close.shift(1))   
price['rtn_1']=price.groupby('code').rtn.shift(1)
###혹시나 모를 오류 제거, 상한가 하한가 30% 이상 차이나는 종목들 제거
price=price.loc[(price['rtn']<=0.3) & (price['rtn']>=-0.3)]

ff=pd.read_csv('ff.csv',index_col=0)
close=pd.pivot_table(price[['date','code','adj_close']],index='date',values='adj_close',columns=['code'],aggfunc='first',dropna=False)


monthend=list()

days=sorted(list(set(ff.index)))

for day in range(len(days)-1):
    if str(days[day])[5:7]!=str(days[day+1])[5:7]:
        monthend.append(days[day])

In [7]:
ff=ff.loc[monthend]
close=close.loc[pd.to_datetime(ff.index)]
close

code,A000020,A000030,A000040,A000050,A000060,A000070,A000075,A000080,A000087,A000100,...,A950200,A950210,A950220,IKQ001,IKQ150,IKR030,IKR100,IKR300,IKS001,IKS200
date,,,,,,,,,,,,,,,,,,,,,
2018-06-29,10200.0,16300.0,2432.0,13850.0,19200.0,119000.0,54390.0,19650.0,16960.0,34910.0,...,NaN,NaN,NaN,818.0,1392.0,7605.0,4831.0,1395.0,2326.0,300.0
2018-07-27,10650.0,16850.0,2380.0,12900.0,18650.0,113000.0,53482.0,18050.0,14865.0,35713.0,...,NaN,NaN,NaN,774.0,1263.0,7565.0,4815.0,1376.0,2295.0,298.0
2018-08-31,11600.0,16300.0,2368.0,13750.0,18250.0,102000.0,55096.0,16800.0,14865.0,38361.0,...,NaN,NaN,NaN,817.0,1376.0,7600.0,4836.0,1393.0,2323.0,300.0
2018-09-28,11350.0,16900.0,2364.0,13000.0,19350.0,101500.0,55702.0,16850.0,14666.0,38923.0,...,NaN,NaN,NaN,822.0,1412.0,7552.0,4829.0,1398.0,2343.0,301.0
2018-10-26,8370.0,16150.0,2340.0,11000.0,18600.0,88300.0,49445.0,15950.0,13817.0,31700.0,...,NaN,NaN,NaN,663.0,1148.0,6551.0,4250.0,1216.0,2027.0,263.0
2018-11-30,9110.0,15650.0,2065.0,11250.0,21200.0,87500.0,49748.0,17500.0,14815.0,36515.0,...,NaN,NaN,NaN,696.0,1187.0,6738.0,4368.0,1257.0,2097.0,271.0
2018-12-28,9100.0,15600.0,2017.0,10800.0,21850.0,78500.0,47831.0,16600.0,14466.0,34301.0,...,NaN,NaN,NaN,676.0,1166.0,6520.0,4210.0,1215.0,2041.0,262.0
2019-01-25,9980.0,NaN,2158.0,11200.0,22300.0,84100.0,48436.0,16300.0,14466.0,39417.0,...,NaN,NaN,NaN,711.0,1168.0,6986.0,4528.0,1298.0,2178.0,282.0
2019-02-28,9800.0,NaN,2045.0,11250.0,22100.0,84300.0,50253.0,17700.0,14566.0,43694.0,...,NaN,NaN,NaN,731.0,1235.0,7133.0,4568.0,1313.0,2195.0,284.0


In [8]:
ff_rtn=ff.pct_change()
ff_rtn=ff_rtn.iloc[1:]
rtn_monthly=close.pct_change()
rtn_monthly=rtn_monthly.iloc[1:]



In [9]:
cp=pd.read_csv('cp.csv',index_col=0,parse_dates=True)
cp=cp/272*21
cp=cp/100
cp=cp.loc[ff_rtn.index]
mkt=pd.read_csv('rf.csv',index_col=0)
mkt.index=cp.index
ff_rtn['mkt']=mkt
ff_rtn['rf']=cp
ff_rtn['mkt_rf']=ff_rtn['mkt']-ff_rtn['rf']
ff_rtn.index=rtn_monthly.index

In [10]:
from tqdm import tqdm
ind=list(ff_rtn.index)
num=15
resid_df=pd.DataFrame(columns=rtn_monthly.columns,index=ind)

####회귀분석 해서 잔차를 구함
for i in tqdm(range(0,len(ind)-num)):
    for code in rtn_monthly.columns:
        y=rtn_monthly.loc[ind[i]:ind[i+num],code]-ff_rtn.loc[ind[i]:ind[i+num],'rf']
        x=ff_rtn.loc[ind[i]:ind[i+num],['mkt_rf','hml','smb']].values
        x=sm.add_constant(x)
        try:
            model=sm.OLS(y,x).fit()
            resid=model.resid
        except Exception as e:
            resid=np.nan
        resid_df.loc[ind[i]:ind[i+num],code]=resid
            
window=12
scaled_resid_df=(resid_df.rolling(window=window).mean())/(resid_df.rolling(window=window).std())
scaled_resid_df=scaled_resid_df.dropna(how='all')
scaled_resid_df=scaled_resid_df.loc[:,scaled_resid_df.columns.str.startswith('A')]

#scaled_resid_df=scaled_resid_df.shift(1)

  0%|          | 0/44 [00:00<?, ?it/s]

100%|██████████| 44/44 [03:38<00:00,  4.97s/it]


In [ ]:
#scaled_resid_df=scaled_resid_df.shift(1)

In [12]:
scaled_resid_df2=scaled_resid_df.stack().reset_index()
scaled_resid_df2.columns=['date','code','scaled_resid']
scaled_resid_df2['date']=pd.to_datetime(scaled_resid_df2['date'])
data=price.set_index(['date','code']).join(scaled_resid_df2.set_index(['date','code']))

data=data.reset_index()

C:\Users\exp202207\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexes\base.py:327: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return libjoin.left_join_indexer_unique(sv, ov)


In [50]:
"""
월말에 얻은 error term을 바로 다음날에 진입하여 한달동안 보유한다고 가정
"""


def backtest(p, width,month_end=ind,temp_data=data,vw=False):
    ###외부상장 종목 제거
    temp_data=temp_data.loc[(temp_data['market']!='외감') & (temp_data['market']!='KONEX')]
    ###거래정지 종목 제거
    temp_data=temp_data.loc[temp_data['market']!='index']
    temp_data=temp_data.loc[temp_data['trading_suspension']!=1]
    ##상장종목수 0인거 제거
    temp_data['listed_stocks'].fillna(0,inplace=True)
    temp_data=temp_data.loc[temp_data['listed_stocks']!=0]
    temp_data['rtn']=np.log(temp_data.groupby('code').adj_close.shift(0)/temp_data.groupby('code').adj_close.shift(1))
    temp_data['rtn_1']=temp_data.groupby('code').rtn.shift(1)
    ###혹시나 모를 오류 제거, 상한가 하한가 30% 이상 차이나는 종목들 제거
    temp_data=temp_data.loc[(temp_data['rtn']<=0.3) & (temp_data['rtn']>=-0.3)]
    daily_rtn=pd.pivot_table(temp_data[['date','code','rtn']],index='date',values='rtn',columns=['code'],aggfunc='first',dropna=False)
    market_cap=pd.pivot_table(temp_data[['date','code','market_cap']],index='date',values='market_cap',columns=['code'],aggfunc='first',dropna=False)
    market_cap=market_cap.shift(1)
    factor_df1=pd.pivot_table(temp_data[['date','code','scaled_resid']],index='date',values='scaled_resid',columns=['code'],aggfunc='first',dropna=False)
    factor_df1=factor_df1.fillna(method='ffill')
    #rebalance_rtn=rebalance_rtn.reindex(factor_df1.index)

    monthly_cap=market_cap.reindex(factor_df1.index)
    return_tickers=dict()
    
    
    
    #daily_rtn=daily_rtn.loc['2004-05':]
    daily_index=list(daily_rtn.index)
    # monthly_index=list(factor_df1.index)
    
    factor_df1=factor_df1.loc[month_end]

    monthly_cap=market_cap.reindex(factor_df1.index)
    for i in range(0,len(factor_df1)-1):
        top_cap=monthly_cap.iloc[i].dropna().rank(ascending=False,method='first')\
            [(monthly_cap.iloc[i].dropna().rank(ascending=False,method='first')>=0)&\
            (monthly_cap.iloc[i].dropna().rank(ascending=False,method='first')<500)].index.values###시총 사위 500개 종목
        
        top_cap=list(top_cap)
        temp_factor1=factor_df1[top_cap]
        
        #temp_factor1=temp_factor1.iloc[i].dropna()
        ####아웃라이어 제거
        num=0.05
        first_sort=temp_factor1.iloc[i].dropna().rank(ascending=True,method='first')\
            [(temp_factor1.iloc[i].dropna().rank(ascending=True,method='first')>(len(temp_factor1.iloc[i].dropna())*num))&\
            (temp_factor1.iloc[i].dropna().rank(ascending=True,method='first')<len(temp_factor1.iloc[i].dropna())*(1-num))]
        
        first_sort=first_sort.dropna().rank(ascending=True,method='first')\
            [(first_sort.dropna().rank(ascending=True,method='first')>len(temp_factor1.iloc[i].dropna())*(p-width))&\
            (first_sort.dropna().rank(ascending=True,method='first')<len(temp_factor1.iloc[i].dropna())*p)].index.values
        first_sort=list(first_sort)
        
            
        start=daily_index.index(factor_df1.index[i])
        if i!=0:
            temp_daily=daily_rtn.loc[daily_index[start+1]:factor_df1.index[i+1]]
        else:
            temp_daily=daily_rtn.loc[:factor_df1.index[i]]
        
        if vw==True:
            
            cap=monthly_cap.iloc[i]
            cap=cap.reindex(monthend)
            cap=cap.dropna()
            
            
            for day in temp_daily.index:
                return_tickers[day] = \
                [[np.dot(daily_rtn.loc[day].reindex(cap.index).values, cap/np.sum(cap))], cap.index]
        
        
        
        else:
            for day in temp_daily.index:
            
                return_tickers[day] = \
                [[np.nanmean(daily_rtn.loc[day].reindex(first_sort).dropna().values )], first_sort]
                #print(np.nansum(daily_rtn.loc[day].reindex(monthend).values * np.array([1/len(monthend) for i in range(len(monthend))])))
    
    
    
    return return_tickers

In [51]:
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
portfolio_vw=pd.DataFrame()

ps=[0.2,0.4,0.6,0.8, 1]
for p in tqdm(ps):

    portfolio_temp=backtest(temp_data=data,p=p,month_end=ind,width=0.2)
    portfolio_vw[f'resid{int(p*5)}th']=pd.Series(np.array([rtn[0] for rtn in list(portfolio_temp.values())]).flatten(), index = portfolio_temp.keys())
    
# portfolio_temp=turnover_momentum(monthly_rtn=monthly_rtn,monthly_turnover=monthly_turnover,q=0.2,l_s=True)
# portfolio[f'short_term_momentum l_s']=pd.Series(np.array([rtn[0] for rtn in list(portfolio_temp.values())]).flatten(), index = portfolio_temp.keys())


#portfolio_vw['l-s']=portfolio_vw['resid5th']-portfolio_vw['resid1th']    
fig=px.line((1+portfolio_vw.dropna(how='all')).cumprod())
fig.show()


#result=portfolio_vw.loc[pd.to_datetime('2016-01-01'):pd.to_datetime('2023-03-01')]

100%|██████████| 5/5 [00:42<00:00,  8.53s/it]


In [52]:
(np.prod(portfolio_vw + 1) ** (252 / portfolio_vw.shape[0]) - 1)/(np.std(portfolio_vw)*np.sqrt(252))

resid1th    1.046217
resid2th    0.041984
resid3th   -0.377380
resid4th   -0.868574
resid5th   -1.273328
dtype: float64

In [55]:
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
portfolio_vw=pd.DataFrame()

ps=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for p in tqdm(ps):

    portfolio_temp=backtest(temp_data=data,p=p,month_end=ind,width=0.1)
    portfolio_vw[f'resid{p*10}th']=pd.Series(np.array([rtn[0] for rtn in list(portfolio_temp.values())]).flatten(), index = portfolio_temp.keys())
    
# portfolio_temp=turnover_momentum(monthly_rtn=monthly_rtn,monthly_turnover=monthly_turnover,q=0.2,l_s=True)
# portfolio[f'short_term_momentum l_s']=pd.Series(np.array([rtn[0] for rtn in list(portfolio_temp.values())]).flatten(), index = portfolio_temp.keys())


#portfolio_vw['l-s']=portfolio_vw['resid5th']-portfolio_vw['resid1th']    
fig=px.line((1+portfolio_vw.dropna(how='all').fillna(0)).cumprod())
fig.show()

100%|██████████| 10/10 [01:23<00:00,  8.39s/it]


In [57]:
def get_mdd(x):
    """
    MDD(Maximum Draw-Down)
    :return: (peak_upper, peak_lower, mdd rate)
    """
    arr_v = np.array(x.fillna(method='ffill'))
    peak_lower = np.argmax(np.maximum.accumulate(arr_v) - arr_v)
    peak_upper = np.argmax(arr_v[:peak_lower])
    return  (arr_v[peak_lower] - arr_v[peak_upper]) / arr_v[peak_upper]



for col in portfolio_vw.columns:
    arr=(1+portfolio_vw.dropna(how='all')[col]).cumprod()
    print(col)
    print(get_mdd(arr))
    print()


resid1.0th
-0.21852218409363902

resid2.0th
-0.29383971802801745

resid3.0th
-0.39332482981096406

resid4.0th
-0.4002029138424358

resid5.0th
-0.45610646491489787

resid6.0th
-0.4679754707160778

resid7.0th
-0.6127103199373691

resid8.0th
-0.6880120145437081

resid9.0th
-0.811803407977238

resid10th
-0.7365197980961546



In [58]:
(np.prod(portfolio_vw + 1) ** (252 / portfolio_vw.shape[0]) - 1)/(np.std(portfolio_vw)*np.sqrt(252))

resid1.0th    1.484468
resid2.0th    0.599321
resid3.0th    0.165223
resid4.0th   -0.086163
resid5.0th   -0.281626
resid6.0th   -0.453617
resid7.0th   -0.776577
resid8.0th   -0.939276
resid9.0th   -1.264993
resid10th    -0.610447
dtype: float64